In [4]:
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers, models
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
from sklearn.preprocessing import StandardScaler


import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

'''
register_matplotlib_converters()
sns.set(style="whitegrid", palette= "muted", font_scale =1.5)
rcParams['figure.figsize']=22,10
'''

'\nregister_matplotlib_converters()\nsns.set(style="whitegrid", palette= "muted", font_scale =1.5)\nrcParams[\'figure.figsize\']=22,10\n'

In [5]:
device_info = tf.config.experimental.list_physical_devices('GPU')

if device_info:
    print("GPU is available. Details:")
    for device in device_info:
        print(device)
else:
    print("No GPU found. Using CPU.")

No GPU found. Using CPU.


In [ ]:
df_train = pd.DataFrame(np.load("./data/train_data.npy"))
df_test =  pd.DataFrame(np.load("./data/test_data.npy"))
y_df_test =  pd.DataFrame(np.load("./data/test_labels.npy"))

df_train.head()

FileNotFoundError: [Errno 2] No such file or directory: './data/test_label.npy'

In [ ]:
def data_engineering(df):
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(df)
    scaled_df = pd.DataFrame(scaled_data)
    return scaled_df, scaler


df_train, train_scaler = data_engineering(df_train)
df_test, _ = data_engineering(df_test)

In [ ]:
df_train = df_train.values.reshape((df_train.shape[0], df_train.shape[1], 1))
df_test = df_test.values.reshape((df_test.shape[0], df_test.shape[1], 1))

In [ ]:
# LSTM model
'''
def get_model(input_shape):
    model = keras.Sequential()

    model.add(keras.layers.LSTM(units=128, input_shape=input_shape))
    model.add(keras.layers.Dropout(rate=0.2))

    model.add(keras.layers.RepeatVector(n=input_shape[0]))
    model.add(keras.layers.LSTM(units=128, return_sequences=True))

    model.add(keras.layers.Dropout(rate=0.2))
    model.add(keras.layers.TimeDistributed(keras.layers.Dense(units=input_shape[1])))

    model.compile(loss='mae', optimizer='adam')
    return model

'''

def get_lstm_autoencoder(input_shape):
    model = models.Sequential()

    # Encoder part: LSTM layers to learn the input sequence
    model.add(layers.LSTM(units=64, activation='relu', input_shape=input_shape, return_sequences=False))
    model.add(layers.RepeatVector(input_shape[0]))  # Repeat the encoded vector for the decoder

    # Decoder part: LSTM layers to reconstruct the original sequence
    model.add(layers.LSTM(units=64, activation='relu', return_sequences=True))
    model.add(layers.TimeDistributed(layers.Dense(units=input_shape[1])))  # Reconstruct the sequence
    
    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    return model

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 64)                16896     
                                                                 
 repeat_vector_3 (RepeatVec  (None, 51, 64)            0         
 tor)                                                            
                                                                 
 lstm_7 (LSTM)               (None, 51, 64)            33024     
                                                                 
 time_distributed_3 (TimeDi  (None, 51, 1)             65        
 stributed)                                                      
                                                                 
Total params: 49985 (195.25 KB)
Trainable params: 49985 (195.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
input_shape = (df_train.shape[1], df_train.shape[2])
# model = get_model(input_shape)
model = get_lstm_autoencoder(input_shape)

In [ ]:
# Training
history = model.fit(
    df_train,
    df_train,
    epochs=2, # 50
    #batch_size=64,
    #validation_split=0.2,
    #shuffle=True
)

Epoch 1/2


2024-11-16 01:28:09.245230: W tensorflow/core/framework/op_kernel.cc:1827] UNKNOWN: JIT compilation failed.
2024-11-16 01:28:09.245287: W tensorflow/core/framework/op_kernel.cc:1827] UNKNOWN: JIT compilation failed.
2024-11-16 01:28:09.245361: W tensorflow/core/framework/op_kernel.cc:1827] UNKNOWN: JIT compilation failed.


UnknownError: Graph execution error:

Detected at node sequential_3/lstm_6/while/lstm_cell/Sigmoid defined at (most recent call last):
  File "/home/cgd/anaconda3/envs/tf215/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/home/cgd/anaconda3/envs/tf215/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/cgd/anaconda3/envs/tf215/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/cgd/anaconda3/envs/tf215/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/cgd/anaconda3/envs/tf215/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/cgd/anaconda3/envs/tf215/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/home/cgd/anaconda3/envs/tf215/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/home/cgd/anaconda3/envs/tf215/lib/python3.10/asyncio/base_events.py", line 1909, in _r

In [ ]:
test_predictions = model.predict(df_test)

# Visualization
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()

14060/14060 [==============================] - 38s 3ms/step


NameError: name 'history' is not defined